In [1]:
import sys
import os
sys.path.append("/home/pi/EE123/Chirp-EE123")

cur_path = os.path.dirname('/home/pi/EE123/Chirp-EE123')
sys.path.insert(0, '/home/pi/EE123/Chirp-EE123/src')

from utils.EE123TNC import *
from utils.hamming import *
from compression.compressData import *

import sounddevice as sd
import RPi.GPIO as GPIO

import matplotlib.pyplot as plt

import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

array_1d_int = npct.ndpointer(dtype=np.int, ndim=1, flags='CONTIGUOUS')

libcd = npct.load_library("../utils/libpll", ".")
libcd.pll.restype = c_int
libcd.pll.argtypes= [array_1d_int, c_int, array_1d_int,array_1d_int,  array_1d_int,array_1d_int, c_int, c_float]

Loading BokehJS ...

# Record

In [26]:
Qin = Queue.Queue()

def queueREPLAY_callback(indata,outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata
    Qin.put( indata.copy()[:,0] ) # Global queue
st = sd.Stream(device=(usb_idx,builtin_idx),callback=queueREPLAY_callback)

st.start()

Stop after TX has finished

In [27]:
st.stop()
st.close()

# Decode and reconstruct image/gif

In [28]:
print("decoding packets")
starttime = time.time()

f1=open('../../asset/buffer.zip','wb')
#f1=open('../../asset/buffer.txt','wb')
f1.truncate()

fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 10)

while(not(Qin.empty())):
    message=Qin.get()
    packets=modem.processBuffer(message)
    for packet in packets:
        ax = modem.decodeAX25(packet)
        f1.write(ax.info)
while not(Qin.empty()):
    samples=Qin.get()

print("DONE!!")
print(time.time() - starttime)
f1.close()

decoding packets
DONE!!
21.627797842025757


In [30]:
import zipfile
import zlib

zf_extract = zipfile.ZipFile("../../asset/buffer.zip")
data = zf_extract.read("../../asset/buffer.txt")
zf_extract.close()

BadZipFile: File is not a zip file

In [ ]:
f = open("../../asset/buffer.txt", "wb")
f.write(data)
f.close()

In [25]:
from PIL import Image

compressR = CompressData("buffer.txt")
result, frame_rate = compressR.decompress()

compressT = CompressData("simpson.png")
compressT.compress()
ori_Data = compressT.image_stack

PSNR = psnr(ori_Data, result)
print("psnr of comression: %.4f" % PSNR)

npArray_play(result, frame_rate = frame_rate)

# path = "rx_simpson.png"
# a = Image.open(path)
# plt.imshow(a)

# !ls -ltr *simpson.png

209280
psnr of comression: 20.4032
